In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
!pip install numpy pandas scipy scikit-learn

In [5]:
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [6]:
# Đọc dữ liệu từ các file đã mở rộng
phobert_df = pd.read_csv('/content/drive/MyDrive/TieuLuan/model/output_phobert.csv')
resnet_df = pd.read_csv('/content/drive/MyDrive/TieuLuan/model/output_resnet50.csv')
cnn3d_df = pd.read_csv('/content/drive/MyDrive/TieuLuan/model/output_3dcnn.csv')

In [73]:
print("Shape dữ liệu từ tập test:")
print("PhoBERT:", phobert_df.shape)
print("ResNet:", resnet_df.shape)
print("3DCNN:", cnn3d_df.shape)

Shape dữ liệu từ tập test:
PhoBERT: (156, 5)
ResNet: (156, 5)
3DCNN: (156, 5)


In [64]:
# Chuyển đổi logits thành xác suất cho PhoBERT và 3DCNN
phobert_probs = softmax(phobert_df.values, axis=1)
resnet_probs = resnet_df.values  # ResNet đã là xác suất
cnn3d_probs = softmax(cnn3d_df.values, axis=1)

In [65]:
# Trọng số dựa trên độ chính xác của từng mô hình
weights = np.array([0.8718, 0.7403, 0.6558])
weights = weights / weights.sum()

In [66]:
# Kết hợp dự đoán từ ba mô hình
ensemble_probs = (weights[0] * phobert_probs +
                  weights[1] * resnet_probs +
                  weights[2] * cnn3d_probs)

In [67]:
# Lấy nhãn dự đoán cuối cùng
ensemble_preds = np.argmax(ensemble_probs, axis=1)

In [68]:
# Tạo nhãn thực tế theo phân phối trong tập test
y_true = np.concatenate([
    np.zeros(31),      # Bình thường
    np.ones(22),       # Kinh dị
    np.full(31, 2),    # Xúc phạm
    np.full(32, 3),    # Phản cảm
    np.full(40, 4)     # Bạo lực
])

In [69]:
# Tính các metrics đánh giá
accuracy = accuracy_score(y_true, ensemble_preds)
precision = precision_score(y_true, ensemble_preds, average='weighted')
recall = recall_score(y_true, ensemble_preds, average='weighted')
f1 = f1_score(y_true, ensemble_preds, average='weighted')

In [48]:
# In kết quả đánh giá
print("\nKết quả Ensemble:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Kết quả Ensemble:
Accuracy: 0.7372
Precision: 0.7623
Recall: 0.7372
F1-score: 0.7418


In [49]:
# In phân phối dự đoán
print("\nPhân phối dự đoán:")
unique, counts = np.unique(ensemble_preds, return_counts=True)
for label, count in zip(unique, counts):
    print(f"Nhãn {label}: {count} mẫu")


Phân phối dự đoán:
Nhãn 0: 41 mẫu
Nhãn 1: 21 mẫu
Nhãn 2: 26 mẫu
Nhãn 3: 24 mẫu
Nhãn 4: 44 mẫu


In [50]:
# Lưu kết quả ensemble
columns = ['0', '1', '2', '3', '4']
pd.DataFrame(ensemble_probs, columns=columns).to_csv('/content/drive/MyDrive/TieuLuan/model/ensemble_predictions.csv', index=False)